In [ ]:
#### 데이터 변수 간략 설명
- Target variable

In [ ]:
---
# 데이터 파악(전처리) 및 EDA
---

In [ ]:
#### 1. data dtypes
- dtype별로 컬럼 리스트 저장

In [ ]:
'''코드작성'''
'''integer 데이터'''
c = train_df.select_dtypes('int64')
integer_var = c.columns.tolist()
inte = pd.DataFrame()
for i in range(len(c.columns)):
    c0 = c.iloc[:,i].unique().tolist()
    
    inte0 = pd.DataFrame({'col_nm':[c.columns[i]],
                        'inte_data':[c0],
                        'inte_num':[len(c.iloc[:,i].unique())]})
    
    inte = inte.append(inte0)
    inte.index = range(len(inte))

In [ ]:
'''float데이터'''
b = train_df.select_dtypes('float64')
numeric_var = b.columns.tolist()

In [ ]:
'''categorical데이터'''
a = train_df.select_dtypes('object')
cate_var = a.columns.tolist()
cate = pd.DataFrame()
for i in range(len(a.columns)):
    a0 = a.iloc[:,i].unique().tolist()
    
    cate0 = pd.DataFrame({'col_nm':[a.columns[i]],
                        'cate_data':[a0],
                        'cate_num':[len(a.iloc[:,i].unique())]})
    
    cate = cate.append(cate0)
    cate.index = range(len(cate))

In [ ]:
#### 2. 결측치 파악 및 처리 방안 구상
- 전체 로우 데이터 수를 구하고 NA %로 컬럼 분할 / 성격이 비슷한 변수별 컬럼 분할
    - 90% 이상 결측치 : 삭제
    - 40-90 : 명목형(missing_value), 연속형(-999)
    - 1-40 : 명목형(최빈값), 연속형(평균값)
    - 단, boosting 모델로 예측하는 모델링을 진행할 것이므로 결측치는 모두 -999로 치환해도 됨
- matshow (결측치 패턴 탐색)
- 연속형 : 회귀모델로 결측치 예측

In [ ]:
'''na값이 아닌 것에 대한 탐색 : NAN값으로 되어 있지 않고 문자열처럼 되어 있는 결측치'''

In [ ]:
'''-999치환'''
df = df.fillna(-999,inplace=True)

In [ ]:
'''결측치 percentage'''
idx = []
for i in range(len(df.isna().sum().tolist())):
    if df.isna().sum().tolist()[i]>0:
        idx.append(i)
df_na_per0 = df.iloc[:,idx]
per0 = pd.Serise(df_na_per0.isna().sum().tolist())/len(df_na_per0)

df_na_per = pd.DataFrame()
df_na_per['NA_column'] = df_na_per0.columns.tolist()
df_na_per['NA_percentage'] = per0

In [ ]:
'''matshow로 결측치 패턴 탐색'''
df = train_transaction[train_transaction.columns[55:].tolist()].isna().astype(int)

plt.figure(figsize=(15,5))
plt.matshow(df,fignum=1)
plt.gca().set_aspect('auto')
plt.show()

In [ ]:
#### 3. data summary
- 이상치 파악 (평균 / 중앙값 비교) : boxplot
- 전체적인 summary값 파악

In [ ]:
---
# Feature EDA
---

In [ ]:
#### categorical variable
- 빈도
- category변수별로 인사이트 뽑을만한 numeric변수의 차이
- 카테고리별로 타겟변수의 분포차이(boxplot)

In [ ]:
inte_low10 = [inte['col_nm'].iloc[i] if inte['inte_num'].iloc[i]<=10 for i in range(len(inte))]
cate_low10 = [cate['col_nm'].iloc[i] if cate['cate_num'].iloc[i]<=10 for i in range(len(cate))]

In [ ]:
'''count : integer,category value <10'''
def count_eda(data,column):
    idx0 = data[column].value_counts(dropna=False).index
    idx = data[column].value_counts().index.tolist()
    value = data[column].value_counts(dropna=False).values
    
    if math.isnan(idx0.tolist()[0]) == True:
        idx = ['nan'] + idx
        
    idx2 = []
    for i in idx:
        idx2.append(str(i))
        
    plt.barh(idx2,value)
    plt.title(column + '변수 빈도수')
    plt.show()

In [ ]:
'''타겟변수별 카테고리 변수값이 빈도 차이'''
'''코드작성'''


In [ ]:
#### numeric variable
- kde
- Target변수와의 상관성 및 분포 확인

In [ ]:
'''커널밀도그래프'''
def kde_target(variable):
    global df
    
    cor = df['isFraud'].corr(df[variable])
    
    # 대출을 상환한 사람과 상환하지 않은 사람의 중앙값 계산
    
    avg_repaid = df.ix[df['target']==0,variable].median()
    avg_not_repaid = df.ix[df['target']==1,variable].median()
    
    plt.figure(figsize=(12,8))
    
    #커널 밀도 그래프 
    sns.kdeplot(df.ix[df['target']==0,variable],label = 'TARGET == 0')
    sns.kdeplot(df.ix[df['target']==1,variable],label = 'TARGET == 1')
    
    plt.xlabel(variable); plt.ylabel('Density'); plt.title('%s Distribution' % variable)
    plt.legend();
    plt.show()
    
    # 상관계수 출력
    # print out the correlation
    print('The correlation between %s and the TARGET is %0.4f' % (variable, cor))
    
    # 중간값 출력
    # Print out average values
    print('Median value for loan that was not repaid = %0.4f' % avg_not_repaid)
    print('Median value for loan that was repaid =     %0.4f' % avg_repaid)


In [ ]:
'''target변수와의 상관성'''


In [ ]:
'''카테고리별로 타겟변수의 분포차이'''
def float_boxplot(variable,df = train_transaction):
    plt.figure(figsize=(12,8))
    sns.boxplot(x = 'isFraud',
               y = variable,
               data= df)
    plt.show()

In [ ]:
---
# FE
---

In [ ]:
#### 필요한 데이터셋 구축

In [ ]:
'''append function'''
def feature_append(target_df,made_feature):
    virtual_df = pd.concat([target_df,made_feature],axis=1)
    
    return virtual_df

In [ ]:
'''기존 변수(변형 금지)'''
train_df

'''파생변수 append하는 변수 - target_df'''
target_df


In [ ]:
#### 대표값 위주의 다양한 파생변수 생성

In [ ]:
'''transform변수를 사용한 mean / median / scale(직접구하는) / std / log / datetime처리 
클러스터변수 / PCA(변수가 너무 많다면) / 변동계수 / 분산 /  ratio / count / split
data encoding'''

In [ ]:
'''cluter - 결측치가 일정개수 미만인 컬럼만으로 비지도학습진행'''
clus_df0 = train_df.select_dtypes('float64')
#clus_df0 = pd.concat([b,train_df.loc[:,['DAYS_BIRTH','DAYS_EMPLOYED','DAYS_ID_PUBLISH']]],axis=1)

no_na0 = clus_df0.isnull().sum().tolist()
no_na = []
for i in range(len(clus_df0.columns)):
    if no_na0[i]<=50000:
        no_na.append(i)
b_df_notnull = clus_df0.iloc[:,no_na]

for i in range(len(b_df_notnull.columns)):
    mn =b_df_notnull.iloc[:,i].mean()
    b_df_notnull.iloc[:,i] = b_df_notnull.iloc[:,i].fillna(mn)

In [ ]:
ks = range(1,10)

inertias = []
for k in ks:

    model = KMeans(n_clusters=k)

    model.fit(b_df_notnull)

    inertias.append(model.inertia_)

In [ ]:
plt.plot(ks, inertias, '-o')
plt.xlabel('number of clusters, k')
plt.ylabel('inertia')
plt.xticks(ks)
plt.show()

In [ ]:
model = KMeans(n_clusters=4,algorithm='auto')
model.fit(b_df_notnull)
predict = pd.DataFrame(model.predict(b_df_notnull))

b_df_notnull['cluster'] = predict.iloc[:,0]

target_df = feature_append(target_df,b_df_notnull[['cluster']])

In [ ]:
'''PCA'''
pca = PCA(n_components=30, random_state=4)

pca_test_df = train_df.iloc[:,41:]
pca_test_df2 = (minmax_scale(pca_test_df, feature_range=(0,1)))
principalComponents = pca.fit_transform(pca_test_df2)

principalDf = pd.DataFrame(principalComponents)

principalDf.rename(columns=lambda x: str(prefix)+str(x), inplace=True)

df = pd.concat([df, principalDf], axis=1)

In [ ]:
'''각 행별 누적합계 구하기
최적의 주성분 개수구하기
n_components30으로 확인한 결과 주성분 9개로 데이터를 95% 설명할 수 있음
but 아래코드에서는 30개까지 사용'''
pca_r = np.cumsum(pca.explained_variance_ratio_)
pca_r

In [ ]:
'''label_encoding'''
for col in train.columns:
    if train[col].dtype == 'object':
        le = LabelEncoder()
        le.fit(list(train[col].astype(str).values) + list(test[col].astype(str).values))
        train[col] = le.transform(list(train[col].astype(str).values))
        test[col] = le.transform(list(test[col].astype(str).values))

'''frequency_encoding'''
value_counts
'''onghot / dummy encoding - 사용하지 않은 이유'''